In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install beautifulsoup4 

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install google-generativeai 

  Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl.metadata (592 bytes)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/155.4 kB ? eta -:--:--
   --------------- ------------------------ 61.4/155.4 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 155.4/155.4 kB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------- ----------------------- 0.5/1.3 MB 17.2 MB/s eta 0:00:01
   -------------------------------- ------- 1.1/1.3 MB 13.7 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 12.1 MB/s eta 0:00:00
Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl (434 kB)
  Attempting uninstall: 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 25.0 which is incompatible.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.5 which is incompatible.
streamlit 1.30.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.1.2 which is incompatible.


In [7]:
pip install python-dotenv 

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install flask 

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install tqdm

In [1]:
import os
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import google.generativeai as genai
from tqdm import tqdm

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [3]:
ARTICLE_URL = "https://www.toronto.com/things-to-do/heres-what-to-expect-at-this-years-world-famous-distillery-winter-village-in-toronto/article_2bd6c2ca-1e2e-51f3-8cc9-ca0619eefe19.html"

In [7]:
response = requests.get(ARTICLE_URL)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")
# This is a generic way to get the main text.
# It grabs paragraphs inside <article> or main content.
paragraphs = []
for p in soup.select("article p"):
    text = p.get_text(strip=True)
    if text:
        paragraphs.append(text)
if not paragraphs:
    # fallback: try all <p> tags
    for p in soup.find_all("p"):
        text = p.get_text(strip=True)
        if text:
            paragraphs.append(text)
print("Number of paragraphs:", len(paragraphs))
paragraphs[:5]

Number of paragraphs: 48


['The magical winter village will run until Jan. 4, 2026.',
 'The magical winter village will run until Jan. 4, 2026.',
 'A view of the crowd gathered on Trinity Street for the lighting of the Christmas Tree on the first day of The Distillery Winter Village.',
 'It’s officially the most wonderful time of year. Toronto’s Distillery Winter Village has kicked off and there’s no shortage of festive fun on the agenda this holiday season.',
 'Now in its 16th year, the celebration has become a global holiday destination and has even been named one of thebest Christmas marketsin the world in 2025 by CNN.']

In [9]:
os.makedirs("../data", exist_ok=True)
with open("../data/cbc_lilith_article.txt", "w", encoding="utf-8") as f:
    f.write("\n\n".join(paragraphs))

In [11]:
sample_comments = [
    "I had no idea Lilith Fair was such a big deal for women in music. This documentary really opened my eyes.",
    "Honestly, I feel like festivals today are more inclusive than what Lilith Fair tried to do back then.",
    "The 90s music scene was so male-dominated. It's inspiring to see how Lilith Fair pushed back against that.",
    "CBC keeps romanticizing the past. Lilith Fair was nice, but it mostly catered to a specific demographic.",
    "I wish we had a modern version of Lilith Fair that included more diverse genres and identities.",
    "This documentary made me nostalgic. The performances and stories really brought back memories.",
    "It sounds like the industry learned very little since the 90s. Women are still underrepresented on festival lineups.",
    "Great doc, but I wanted more critical discussion of who got left out of the Lilith Fair story.",
    "I love how the festival used its platform for social causes, not just music.",
    "We need more content like this that celebrates women musicians and organizers."
]

df_comments = pd.DataFrame({
    "article_url": ARTICLE_URL,
    "comment_id": range(1, len(sample_comments) + 1),
    "comment_text": sample_comments
})

df_comments


,article_url,comment_id,comment_text
0,https://www.toronto.com/things-to-do/heres-wha...,1,I had no idea Lilith Fair was such a big deal ...
1,https://www.toronto.com/things-to-do/heres-wha...,2,"Honestly, I feel like festivals today are more..."
2,https://www.toronto.com/things-to-do/heres-wha...,3,The 90s music scene was so male-dominated. It'...
3,https://www.toronto.com/things-to-do/heres-wha...,4,CBC keeps romanticizing the past. Lilith Fair ...
4,https://www.toronto.com/things-to-do/heres-wha...,5,I wish we had a modern version of Lilith Fair ...
5,https://www.toronto.com/things-to-do/heres-wha...,6,This documentary made me nostalgic. The perfor...
6,https://www.toronto.com/things-to-do/heres-wha...,7,It sounds like the industry learned very littl...
7,https://www.toronto.com/things-to-do/heres-wha...,8,"Great doc, but I wanted more critical discussi..."
8,https://www.toronto.com/things-to-do/heres-wha...,9,I love how the festival used its platform for ...
9,https://www.toronto.com/things-to-do/heres-wha...,10,We need more content like this that celebrates...


In [13]:
raw_path = "../data/cbc_lilith_comments_raw.csv"
df_comments.to_csv(raw_path, index=False)
raw_path

'../data/cbc_lilith_comments_raw.csv'

In [15]:
df = pd.read_csv("../data/cbc_lilith_comments_raw.csv")
df.head()


,article_url,comment_id,comment_text
0,https://www.toronto.com/things-to-do/heres-wha...,1,I had no idea Lilith Fair was such a big deal ...
1,https://www.toronto.com/things-to-do/heres-wha...,2,"Honestly, I feel like festivals today are more..."
2,https://www.toronto.com/things-to-do/heres-wha...,3,The 90s music scene was so male-dominated. It'...
3,https://www.toronto.com/things-to-do/heres-wha...,4,CBC keeps romanticizing the past. Lilith Fair ...
4,https://www.toronto.com/things-to-do/heres-wha...,5,I wish we had a modern version of Lilith Fair ...


In [17]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text)
    text = re.sub(r"\s+", " ", text)      # normalize spaces
    text = text.replace("“", '"').replace("”", '"').replace("’", "'")
    return text.strip()

df["comment_clean"] = df["comment_text"].apply(clean_text)
df = df[df["comment_clean"].str.len() > 0].reset_index(drop=True)

df[["comment_text", "comment_clean"]]


,comment_text,comment_clean
0,I had no idea Lilith Fair was such a big deal ...,I had no idea Lilith Fair was such a big deal ...
1,"Honestly, I feel like festivals today are more...","Honestly, I feel like festivals today are more..."
2,The 90s music scene was so male-dominated. It'...,The 90s music scene was so male-dominated. It'...
3,CBC keeps romanticizing the past. Lilith Fair ...,CBC keeps romanticizing the past. Lilith Fair ...
4,I wish we had a modern version of Lilith Fair ...,I wish we had a modern version of Lilith Fair ...
5,This documentary made me nostalgic. The perfor...,This documentary made me nostalgic. The perfor...
6,It sounds like the industry learned very littl...,It sounds like the industry learned very littl...
7,"Great doc, but I wanted more critical discussi...","Great doc, but I wanted more critical discussi..."
8,I love how the festival used its platform for ...,I love how the festival used its platform for ...
9,We need more content like this that celebrates...,We need more content like this that celebrates...


In [60]:
SYSTEM_INSTRUCTIONS = """
You are an assistant that classifies public comments responding to a CBC documentary article
about Lilith Fair, a 1990s music festival featuring women artists.

For each comment, you MUST return a STRICT JSON object with:
- sentiment: one of ["positive", "neutral", "negative"]
- stance: one of ["supports_festival", "criticizes_festival", "mixed", "unrelated"]
- toxicity: one of ["none", "low", "medium", "high"]
- topic: a short phrase summarizing the main issue (e.g., "women in music", "nostalgia", "diversity", "industry sexism").
- explanation: 1-2 sentences explaining your labels in plain language.

Return ONLY valid JSON. Do not include code fences or extra commentary.
"""

model = genai.GenerativeModel("models/gemini-2.5-flash")


In [62]:
def parse_json_from_response(text):
    text = text.strip()
    if text.startswith("```"):
        text = text.strip("`")
        text = re.sub(r"^json", "", text, flags=re.IGNORECASE).strip()
    return json.loads(text)


In [64]:
def classify_comment(text):
    prompt = f"""{SYSTEM_INSTRUCTIONS}

Comment: "{text}"

Respond with JSON now."""
    response = model.generate_content(prompt)
    raw = response.text
    try:
        return parse_json_from_response(raw)
    except Exception as e:
        print("Parse error:", e)
        print("Raw:", raw[:300])
        return {
            "sentiment": "neutral",
            "stance": "mixed",
            "toxicity": "none",
            "topic": "unknown",
            "explanation": f"Parsing error: {e}"
        }


In [66]:
results = []
for text in tqdm(df["comment_clean"].tolist()):
    out = classify_comment(text)
    results.append(out)

len(results)


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:29<00:00,  2.99s/it]


10

In [68]:
df_labels = pd.DataFrame(results)
df_labeled = pd.concat([df.reset_index(drop=True),
                        df_labels.reset_index(drop=True)], axis=1)

df_labeled

,article_url,comment_id,comment_text,comment_clean,sentiment,stance,toxicity,topic,explanation
0,https://www.toronto.com/things-to-do/heres-wha...,1,I had no idea Lilith Fair was such a big deal ...,I had no idea Lilith Fair was such a big deal ...,positive,supports_festival,none,impact of Lilith Fair,The comment expresses a positive learning expe...
1,https://www.toronto.com/things-to-do/heres-wha...,2,"Honestly, I feel like festivals today are more...","Honestly, I feel like festivals today are more...",negative,criticizes_festival,none,festival inclusivity,The comment expresses a negative sentiment by ...
2,https://www.toronto.com/things-to-do/heres-wha...,3,The 90s music scene was so male-dominated. It'...,The 90s music scene was so male-dominated. It'...,positive,supports_festival,none,women in music industry,"The comment expresses positive sentiment, call..."
3,https://www.toronto.com/things-to-do/heres-wha...,4,CBC keeps romanticizing the past. Lilith Fair ...,CBC keeps romanticizing the past. Lilith Fair ...,neutral,mixed,none,festival inclusivity and media portrayal,The comment acknowledges a positive aspect ('n...
4,https://www.toronto.com/things-to-do/heres-wha...,5,I wish we had a modern version of Lilith Fair ...,I wish we had a modern version of Lilith Fair ...,positive,supports_festival,none,diversity and modern festivals,The comment expresses a positive desire for a ...
5,https://www.toronto.com/things-to-do/heres-wha...,6,This documentary made me nostalgic. The perfor...,This documentary made me nostalgic. The perfor...,positive,supports_festival,none,nostalgia,The comment expresses positive feelings of nos...
6,https://www.toronto.com/things-to-do/heres-wha...,7,It sounds like the industry learned very littl...,It sounds like the industry learned very littl...,negative,supports_festival,none,women in music,The comment expresses disappointment that the ...
7,https://www.toronto.com/things-to-do/heres-wha...,8,"Great doc, but I wanted more critical discussi...","Great doc, but I wanted more critical discussi...",neutral,mixed,none,representation and inclusivity in festival's n...,The comment praises the documentary but then r...
8,https://www.toronto.com/things-to-do/heres-wha...,9,I love how the festival used its platform for ...,I love how the festival used its platform for ...,positive,supports_festival,none,festival's social impact,The comment expresses strong approval ('I love...
9,https://www.toronto.com/things-to-do/heres-wha...,10,We need more content like this that celebrates...,We need more content like this that celebrates...,positive,supports_festival,none,women in music,"The comment is positive, expressing a desire f..."


In [70]:
processed_path = "../data/cbc_lilith_comments_labeled.csv"
df_labeled.to_csv(processed_path, index=False)
processed_path

'../data/cbc_lilith_comments_labeled.csv'